In [1]:
import SimpleITK as sitk
import glob
import os
import glob
import lpips
import torch
import numpy as np
import pandas as pd
from skimage.metrics import structural_similarity

In [2]:
manifest = pd.read_excel('/mnt/data/home/ZC/denoising/Patient_lists/fixedCT_static_shuffled_batched.xlsx', dtype=str)
manifest = manifest[manifest['batch'] == '5']
manifest = manifest[['batch', 'Patient_ID', 'Patient_subID']].sort_values(by=['Patient_ID', 'Patient_subID'])
manifest

,batch,Patient_ID,Patient_subID
97,5,00010436,0000034479
95,5,00010461,0000455845
99,5,00014689,0000455416
93,5,00019591,0000455445
92,5,00035838,0000455369
85,5,00105734,0000455323
89,5,00154137,0000455529
91,5,00174234,0000455725
94,5,00214792,0000455334
88,5,00214836,0000455414


In [3]:
def read_imgs(manifest, input_folder, filename):
    filenames = []
    for _, row in manifest.iterrows():
        filenames.append(os.path.join(input_folder, row['Patient_ID'], row['Patient_subID'], filename))

    imgs = []

    for i, filename in enumerate(filenames):
        print(i, end=', ', flush=True)
        imgs.append(sitk.GetArrayFromImage(sitk.ReadImage(filename)))

    imgs = np.array(imgs)

    return imgs

In [4]:
def calc_mae_with_cut_off(imgs1, imgs2, vmin, vmax):
    mae_volumes = []

    for img1, img2 in zip(imgs1, imgs2):
        maes = []
        for slice1, slice2 in zip(img1, img2):
            mae = np.mean(np.abs(np.clip(slice1, vmin, vmax) - np.clip(slice2, vmin, vmax)))
            maes.append(mae)
        mae_volumes.append(np.mean(maes))

    return np.mean(mae_volumes), np.std(mae_volumes)

In [5]:
def calc_mae_with_ref_window(imgs, refs, vmin, vmax):
    mae_volumes = []

    for img, ref in zip(imgs, refs):
        maes = []
        for slice_img, slice_ref in zip(img, ref):
            mask = np.where((slice_ref >= vmin) & (slice_ref <= vmax), 1, 0)
            mae = np.sum(np.abs(slice_img - slice_ref) * mask) / np.sum(mask)
            maes.append(mae)
        mae_volumes.append(np.mean(maes))

    return np.mean(mae_volumes), np.std(mae_volumes)

In [6]:
def calc_ssim_with_cut_off(imgs1, imgs2, vmin, vmax):
    ssim_volumes = []
    
    for img1, img2 in zip(imgs1, imgs2):
        ssims = []
        for slice1, slice2 in zip(img1, img2):
            ssim = structural_similarity(np.clip(slice1, vmin, vmax), np.clip(slice2, vmin, vmax), data_range=vmax - vmin)
            ssims.append(ssim)

        ssim_volumes.append(np.mean(ssims))

    return np.mean(ssim_volumes), np.std(ssim_volumes)

In [7]:
def calc_ssim_with_ref_window(imgs, refs, vmin, vmax):
    ssim_volumes = []

    for img, ref in zip(imgs, refs):
        ssims = []
        for slice_img, slice_ref in zip(img, ref):
            mask = np.where((slice_ref >= vmin) & (slice_ref <= vmax), 1, 0)
            _, ssim_map = structural_similarity(slice_img, slice_ref, data_range=vmax - vmin, full=True)
            ssim = np.sum(ssim_map * mask) / np.sum(mask)
            ssims.append(ssim)

        ssim_volumes.append(np.mean(ssims))

    return np.mean(ssim_volumes), np.std(ssim_volumes)

In [8]:
base_dir = '/mnt/data/home/ZC/denoising/models/'

print('Reading ref...', flush=True)
ref_imgs = read_imgs(manifest, os.path.join(base_dir, 'supervised_poisson/pred_images'), 'random_0/epoch58_1/gt_img.nii.gz')
print('Reading noisy...', flush=True)
noisy_imgs = read_imgs(manifest, os.path.join(base_dir, 'supervised_poisson/pred_images'), 'random_0/epoch58_1/condition_img.nii.gz')
print('Reading supervised...', flush=True)
supervised_imgs = read_imgs(manifest, os.path.join(base_dir, 'supervised_poisson/pred_images'), 'random_0/epoch58_1/pred_img.nii.gz')
print('Reading noise2noise diffusion...', flush=True)
n2n_diff_imgs = read_imgs(manifest, os.path.join(base_dir, 'unsupervised_gaussian_current_beta0/pred_images'), 'random_0/epoch61avg/pred_img_scans20.nii.gz')
print('Reading noise2noise...', flush=True)
n2n_imgs = read_imgs(manifest, os.path.join(base_dir, 'noise2noise/pred_images'), 'random_0/epoch78/pred_img.nii.gz')


Reading ref...
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, Reading noisy...
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, Reading supervised...
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, Reading noise2noise diffusion...
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, Reading noise2noise...
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 

In [14]:
def calc_lpips(imgs1, imgs2, vmin, vmax):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    loss_fn = lpips.LPIPS().to(device)
    lpips_volumes = []

    for img1, img2 in zip(imgs1, imgs2):
        lpipss = []
        for slice1, slice2 in zip(img1, img2):
            slice1 = np.clip(slice1, vmin, vmax).astype(np.float32)
            slice2 = np.clip(slice2, vmin, vmax).astype(np.float32)

            slice1 = (slice1 - vmin) / (vmax - vmin) * 2 - 1
            slice2 = (slice2 - vmin) / (vmax - vmin) * 2 - 1

            slice1 = np.stack([slice1, slice1, slice1], axis=-1)
            slice2 = np.stack([slice2, slice2, slice2], axis=-1)

            slice1 = np.transpose(slice1, (2, 0, 1))[np.newaxis, ...]
            slice2 = np.transpose(slice2, (2, 0, 1))[np.newaxis, ...]

            slice1 = torch.from_numpy(slice1).to(device)
            slice2 = torch.from_numpy(slice2).to(device)

            lpips_val = loss_fn(slice1, slice2)
            lpipss.append(lpips_val.item())

        lpips_volumes.append(lpipss)

    return np.mean(lpips_volumes), np.std(lpips_volumes)


In [15]:
lpips_noisy, std_noisy = calc_lpips(noisy_imgs, ref_imgs, 0, 100)
print(f'LPIPS Noisy: {lpips_noisy:.4f} ± {std_noisy:.4f}', flush=True)

lpips_n2n, std_n2n = calc_lpips(n2n_imgs, ref_imgs, 0, 100)
print(f'LPIPS N2N: {lpips_n2n:.4f} ± {std_n2n:.4f}', flush=True)

lpips_supervised, std_supervised = calc_lpips(supervised_imgs, ref_imgs, 0, 100)
print(f'LPIPS Supervised: {lpips_supervised:.4f} ± {std_supervised:.4f}', flush=True)

lpips_n2n_diff, std_n2n_diff = calc_lpips(n2n_diff_imgs, ref_imgs, 0, 100)
print(f'LPIPS N2N Diffusion: {lpips_n2n_diff:.4f} ± {std_n2n_diff:.4f}', flush=True)


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/dw640/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
LPIPS Noisy: 0.1514 ± 0.0246
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/dw640/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
LPIPS N2N: 0.0917 ± 0.0146
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/dw640/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
LPIPS Supervised: 0.0635 ± 0.0154
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/dw640/.local/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth
LPIPS N2N Diffusion: 0.0422 ± 0.0100


In [16]:
mae_noisy, std_noisy = calc_mae_with_cut_off(ref_imgs, noisy_imgs, -1000, 2000)
print(f'MAE Noisy: {mae_noisy:.2f} ± {std_noisy:.2f}', flush=True)

mae_n2n, std_n2n = calc_mae_with_cut_off(ref_imgs, n2n_imgs, -1000, 2000)
print(f'MAE Noise2Noise: {mae_n2n:.2f} ± {std_n2n:.2f}', flush=True)

mae_supervised, std_supervised = calc_mae_with_cut_off(ref_imgs, supervised_imgs, -1000, 2000)
print(f'MAE Supervised: {mae_supervised:.2f} ± {std_supervised:.2f}', flush=True)

mae_n2n_diff, std_n2n_diff = calc_mae_with_cut_off(ref_imgs, n2n_diff_imgs, -1000, 2000)
print(f'MAE Noise2Noise Diffusion: {mae_n2n_diff:.2f} ± {std_n2n_diff:.2f}', flush=True)

MAE Noisy: 4.51 ± 0.56
MAE Noise2Noise: 6.89 ± 0.72
MAE Supervised: 4.76 ± 0.59
MAE Noise2Noise Diffusion: 4.53 ± 0.20


In [17]:
mae_noisy, std_noisy = calc_mae_with_ref_window(noisy_imgs, ref_imgs, 0, 100)
print(f'MAE Noisy with ref window: {mae_noisy:.2f} ± {std_noisy:.2f}', flush=True)

mae_n2n, std_n2n = calc_mae_with_ref_window(n2n_imgs, ref_imgs, 0, 100)
print(f'MAE Noise2Noise with ref window: {mae_n2n:.2f} ± {std_n2n:.2f}', flush=True)

mae_supervised, std_supervised = calc_mae_with_ref_window(supervised_imgs, ref_imgs, 0, 100)
print(f'MAE Supervised with ref window: {mae_supervised:.2f} ± {std_supervised:.2f}', flush=True)

mae_n2n_diff, std_n2n_diff = calc_mae_with_ref_window(n2n_diff_imgs, ref_imgs, 0, 100)
print(f'MAE Noise2Noise Diffusion with ref window: {mae_n2n_diff:.2f} ± {std_n2n_diff:.2f}', flush=True)


MAE Noisy with ref window: 6.28 ± 0.59
MAE Noise2Noise with ref window: 4.12 ± 0.40
MAE Supervised with ref window: 3.27 ± 0.44
MAE Noise2Noise Diffusion with ref window: 2.62 ± 0.27


In [18]:
ssim_noisy, std_noisy = calc_ssim_with_cut_off(ref_imgs, noisy_imgs, -1000, 2000)
print(f'SSIM Noisy: {ssim_noisy:.3f} ± {std_noisy:.3f}', flush=True)

ssim_n2n, std_n2n = calc_ssim_with_cut_off(ref_imgs, n2n_imgs, -1000, 2000)
print(f'SSIM Noise2Noise: {ssim_n2n:.3f} ± {std_n2n:.3f}', flush=True)

ssim_supervised, std_supervised = calc_ssim_with_cut_off(ref_imgs, supervised_imgs, -1000, 2000)
print(f'SSIM Supervised: {ssim_supervised:.3f} ± {std_supervised:.3f}', flush=True)

ssim_n2n_diff, std_n2n_diff = calc_ssim_with_cut_off(ref_imgs, n2n_diff_imgs, -1000, 2000)
print(f'SSIM Noise2Noise Diffusion: {ssim_n2n_diff:.3f} ± {std_n2n_diff:.3f}', flush=True)


SSIM Noisy: 0.996 ± 0.001
SSIM Noise2Noise: 0.995 ± 0.001
SSIM Supervised: 0.998 ± 0.000
SSIM Noise2Noise Diffusion: 0.999 ± 0.000


In [19]:
ssim_noisy, std_noisy = calc_ssim_with_ref_window(noisy_imgs, ref_imgs, 0, 100)
print(f'SSIM Noisy with ref window: {ssim_noisy:.3f} ± {std_noisy:.3f}', flush=True)

ssim_n2n, std_n2n = calc_ssim_with_ref_window(n2n_imgs, ref_imgs, 0, 100)
print(f'SSIM Noise2Noise with ref window: {ssim_n2n:.3f} ± {std_n2n:.3f}', flush=True)

ssim_supervised, std_supervised = calc_ssim_with_ref_window(supervised_imgs, ref_imgs, 0, 100)
print(f'SSIM Supervised with ref window: {ssim_supervised:.3f} ± {std_supervised:.3f}', flush=True)

ssim_n2n_diff, std_n2n_diff = calc_ssim_with_ref_window(n2n_diff_imgs, ref_imgs, 0, 100)
print(f'SSIM Noise2Noise Diffusion with ref window: {ssim_n2n_diff:.3f} ± {std_n2n_diff:.3f}', flush=True)

SSIM Noisy with ref window: 0.412 ± 0.053
SSIM Noise2Noise with ref window: 0.693 ± 0.032
SSIM Supervised with ref window: 0.658 ± 0.053
SSIM Noise2Noise Diffusion with ref window: 0.740 ± 0.025
